# HTMLHeaderTextSplitter  

- Author: [ChangJun Lee](https://www.linkedin.com/in/cjleeno1/)
- Design: []() 
- Peer Review: [YooKyung Jeon](https://github.com/sirena1), [Wooseok Jeong](https://github.com/jeong-wooseok)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab] [![Open in GitHub]

## Overview

This is a "structure-aware" chunk generator that splits text at the element level and adds metadata for each header, conceptually similar to the <a href="https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/markdown_header_metadata">`MarkdownHeaderTextSplitter`</a>.

It adds metadata "related" to each chunk.

`HTMLHeaderTextSplitter` can return chunks by element or combine elements with the same metadata,

- (a) semantically (approximately) group related text and
- (b) preserve context-rich information encoded in the document structure.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Using HTML Strings](#using-html-strings)
- [Connecting with Other Splitters and Loading HTML from a Web URL](#connecting-with-other-splitters-and-loading-html-from-a-web-url)
- [Limitations](#limitations)

### References 

- [HTML Header Text Splitter](https://python.langchain.com/api_reference/text_splitters/html/langchain_text_splitters.html.HTMLHeaderTextSplitter.html#)
---

## Environment Setup

Setting up your environment is the first step. See the [Environment Setup](https://wikidocs.net/257836) guide for more details.

**[Note]**
- The `langchain-opentutorial` is a package of easy-to-use environment setup guidance, useful functions and utilities for tutorials.
- Check out the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

## Using HTML Strings


- Specify the header tags and their names to split on in the `headers_to_split_on` list as tuples.
- Create an `HTMLHeaderTextSplitter` object and pass the list of headers to split on to the `headers_to_split_on` parameter.

In [19]:
%%capture --no-stderr
%pip install langchain langchain-opentutorial

In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
    ]
)

In [21]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "RecursiveJsonSplitter",
    }
)

Environment variables have been set successfully.


In [22]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Header 1</h1>
        <p>Text included in Header 1</p>
        <div>
            <h2>Header 2-1 Title</h2>
            <p>Text included in Header 2-1</p>
            <h3>Header 3-1 Title</h3>
            <p>Text included in Header 3-1</p>
            <h3>Header 3-2 Title</h3>
            <p>Text included in Header 3-2</p>
        </div>
        <div>
            <h2>Header 2-2 Title</h2>
            <p>Text included in Header 2-2</p>
        </div>
        <br>
        <p>Last content</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),  # Specify the header tags and their names to split on.
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

# Create an HTMLHeaderTextSplitter object to split the HTML text based on the specified headers.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# Split the HTML string and store the result in the html_header_splits variable.
html_header_splits = html_splitter.split_text(html_string)
# Print the split results.
for header in html_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

Header 1
{}
Text included in Header 1  
Header 2-1 Title Header 3-1 Title Header 3-2 Title
{'Header 1': 'Header 1'}
Text included in Header 2-1
{'Header 1': 'Header 1', 'Header 2': 'Header 2-1 Title'}
Text included in Header 3-1
{'Header 1': 'Header 1', 'Header 2': 'Header 2-1 Title', 'Header 3': 'Header 3-1 Title'}
Text included in Header 3-2
{'Header 1': 'Header 1', 'Header 2': 'Header 2-1 Title', 'Header 3': 'Header 3-2 Title'}
Header 2-2 Title
{'Header 1': 'Header 1'}
Text included in Header 2-2
{'Header 1': 'Header 1', 'Header 2': 'Header 2-2 Title'}
Last content
{'Header 1': 'Header 1'}


## Connecting with Other Splitters and Loading HTML from a Web URL

In this example, we load HTML content from a web URL and then process it by connecting it with other splitters in a pipeline.


In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

url = "https://plato.stanford.edu/entries/goedel/"  # Specify the URL of the text to split.

headers_to_split_on = [  # Specify the HTML header tags and their names to split on.
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

# Create an HTMLHeaderTextSplitter object to split the text based on the specified HTML headers.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Fetch the text from the URL and split it based on the HTML headers.
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500  # Specify the size of the chunks to split the text into.
chunk_overlap = 30  # Specify the number of overlapping characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(  # Create a RecursiveCharacterTextSplitter object to recursively split the text.
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split the text that was split by HTML headers into chunks of the specified size.
splits = text_splitter.split_documents(html_header_splits)

# Print chunks 80 to 85 of the split text.
for header in splits[80:85]:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

We see that Gödel first tried to reduce the consistency problem for analysis to that of arithmetic. This seemed to require a truth definition for arithmetic, which in turn led to paradoxes, such as the Liar paradox (“This sentence is false”) and Berry’s paradox (“The least number not defined by an expression consisting of just fourteen English words”). Gödel then noticed that such paradoxes would not necessarily arise if truth were replaced by provability. But this means that arithmetic truth
{'Header 1': 'Kurt Gödel', 'Header 2': '2. Gödel’s Mathematical Work', 'Header 3': '2.2 The Incompleteness Theorems', 'Header 4': '2.2.1 The First Incompleteness Theorem'}
means that arithmetic truth and arithmetic provability are not co-extensive — whence the First Incompleteness Theorem.
{'Header 1': 'Kurt Gödel', 'Header 2': '2. Gödel’s Mathematical Work', 'Header 3': '2.2 The Incompleteness Theorems', 'Header 4': '2.2.1 The First Incompleteness Theorem'}
This account of Gödel’s discovery was t

## Limitations

HTMLHeaderTextSplitter attempts to handle structural differences between HTML documents, but it may sometimes miss specific headers.

For example, this algorithm assumes that headers are always nodes "above" the related text, i.e., in previous sibling nodes, ancestor nodes, and combinations thereof.

In the following news article (as of the time of writing), the text of the top headline is tagged as "h1", but it is in a **separate subtree** from the text element we expect.

Therefore, the text related to the "h1" element does not appear in the chunk metadata, but the text related to "h2" does, if applicable.


In [24]:
# Specify the URL of the HTML page to split.
url = "https://www.cnn.com/2023/09/25/weather/el-nino-winter-us-climate/index.html"

headers_to_split_on = [
    ("h1", "Header 1"),  # Specify the header tags and their names to split on.
    ("h2", "Header 2"),  # Specify the header tags and their names to split on.
]

# Create an HTMLHeaderTextSplitter object to split the HTML text based on the specified headers.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Split the HTML page from the specified URL and store the result in the html_header_splits variable.
html_header_splits = html_splitter.split_text_from_url(url)

# Print the split results.
for header in html_header_splits:
    print(f"{header.page_content[:100]}")
    print(f"{header.metadata}", end="\n=====================\n")

CNN values your feedback  
1. How relevant is this ad to you?  
2. Did you encounter any technical i
{}
No two El Niño winters are the same, but many have temperature and precipitation trends in common.  
{'Header 2': 'What could this winter look like?'}
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad Feedback  
Ad
{}
